In [1]:
import pandas as pd, numpy as np, os
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
from matplotlib.backends.backend_pdf import PdfPages

In [2]:
output_dir ='/ihme/costeffectiveness/results/vivarium_ciff_sam/v4.5.5_linear_scale-up_etc/ciff_sam/2021_11_23_17_59_09/count_data/'

In [5]:
wasting_pt = pd.read_csv(output_dir +'wasting_state_person_time.csv')
wasting_pt.head()

,Unnamed: 0,sex,year,cause,measure,input_draw,scenario,value,x_factor,sq_lns,mam_treatment,sam_treatment,age
0,0,female,2022,mild_child_wasting,state_person_time,29,baseline,0.0,cat2,covered,covered,covered,early_neonatal
1,1,female,2022,mild_child_wasting,state_person_time,29,baseline,0.0,cat1,covered,covered,covered,early_neonatal
2,2,female,2022,mild_child_wasting,state_person_time,29,baseline,0.0,cat2,covered,uncovered,covered,early_neonatal
3,3,female,2022,mild_child_wasting,state_person_time,29,baseline,0.0,cat1,covered,uncovered,covered,early_neonatal
4,4,female,2022,mild_child_wasting,state_person_time,29,baseline,0.0,cat2,covered,covered,uncovered,early_neonatal


In [6]:
wasting_transitions = pd.read_csv(output_dir + 'wasting_transition_count.csv').drop(columns='Unnamed: 0')
wasting_transitions.head()

,sex,year,measure,input_draw,scenario,value,x_factor,sq_lns,mam_treatment,sam_treatment,age
0,female,2022,mild_child_wasting_to_moderate_acute_malnutrit...,29,baseline,0.0,cat2,covered,covered,covered,early_neonatal
1,female,2022,mild_child_wasting_to_moderate_acute_malnutrit...,29,baseline,0.0,cat1,covered,covered,covered,early_neonatal
2,female,2022,mild_child_wasting_to_moderate_acute_malnutrit...,29,baseline,0.0,cat2,covered,uncovered,covered,early_neonatal
3,female,2022,mild_child_wasting_to_moderate_acute_malnutrit...,29,baseline,0.0,cat1,covered,uncovered,covered,early_neonatal
4,female,2022,mild_child_wasting_to_moderate_acute_malnutrit...,29,baseline,0.0,cat2,covered,covered,uncovered,early_neonatal


# V&V Step 1: wasting remission rate RRs by treatment coverage

Verify the relative risks for MAM and SAM recovery rates by treatment coverage status in the Wasting transition rate relative risks for wasting treatment table. Note that the relative risks will vary between the baseline and alternative scenarios given the reliance of the relative risks on E_MAM and E_SAM values, which vary between simulation scenarios.

## Step 1.1: calculate target RRs

In [151]:
e_mam_baseline = 0.731
e_sam_baseline = 0.7
e_mam_alternative = 0.725
e_sam_alternative = 0.725
r_mam_ux = 365 / 63
r_mam_tx = 365 / 41.3

In [152]:
rr_mam_to_mild_baseline = r_mam_ux / (r_mam_tx * e_mam_baseline + r_mam_ux * (1 - e_mam_baseline))
rr_sam_to_mild_baseline = 0
rr_sam_to_mam_baseline = 1 / (1 - e_sam_baseline)
print(rr_mam_to_mild_baseline, rr_sam_to_mild_baseline, rr_sam_to_mam_baseline)

0.7224991121832943 0 3.333333333333333


In [153]:
rr_mam_to_mild_alternative = r_mam_ux / (r_mam_tx * e_mam_alternative + r_mam_ux * (1 - e_mam_alternative))
rr_sam_to_mild_alternative = 0
rr_sam_to_mam_alternative = 1 / (1 - e_sam_alternative)
print(rr_mam_to_mild_alternative, rr_sam_to_mild_alternative, rr_sam_to_mam_alternative)

0.7241485118134398 0 3.6363636363636362


## Step 1.2: calculate simulation remission RRs

In [154]:
def calculate_wasting_transition_rrs(wasting_transitions, wasting_pt,
                                      ages, transition, source_state, 
                                      groupby_cols):
    if source_state=='moderate_acute_malnutrition':
        sev = 'mam'
    elif source_state=='severe_acute_malnutrition':
        sev = 'sam'
    transition = (wasting_transitions.loc[(wasting_transitions.age.isin(ages))
                                         & (wasting_transitions.measure==transition)]
                  .groupby(groupby_cols).sum())
    pt = (wasting_pt.loc[(wasting_pt.age.isin(ages))
                       & (wasting_pt.cause==source_state)]
          .groupby(groupby_cols).sum())
    transition_rate = transition / pt
    transition_rate_covered = (transition_rate.reset_index().loc[transition_rate.reset_index()[[c for c in groupby_cols if 'treatment' in c][0]]=='covered']
                               .set_index([c for c in groupby_cols if c != [c for c in groupby_cols if 'treatment' in c][0]])
                               .drop(columns=[c for c in groupby_cols if 'treatment' in c][0]))
    rr = transition_rate / transition_rate_covered
    rr = (rr.groupby([c for c in groupby_cols if c != 'input_draw'])
                       .describe(percentiles=[0.025,0.975]))['value']
    return rr

In [155]:
ages = ['6-11_months', '12_to_23_months', '2_to_4']

# mam to mild (r3)
print('MAM to mild RR target baseline:', rr_mam_to_mild_baseline,
      '\nMAM to mild RR target alternative:', rr_mam_to_mild_alternative)
calculate_wasting_transition_rrs(wasting_transitions, wasting_pt, ages, 
                                 'moderate_acute_malnutrition_to_mild_child_wasting_event_count', 
                                 'moderate_acute_malnutrition', 
                                  ['input_draw','scenario','mam_treatment'])

MAM to mild RR target baseline: 0.7224991121832943 
MAM to mild RR target alternative: 0.7241485118134398


count      mean       std       min  \
scenario          mam_treatment                                        
baseline          covered         12.0  1.000000  0.000000  1.000000   
                  uncovered       12.0  0.732549  0.022108  0.695236   
sqlns             covered         12.0  1.000000  0.000000  1.000000   
                  uncovered       12.0  0.722863  0.007948  0.711234   
wasting_treatment covered         12.0  1.000000  0.000000  1.000000   
                  uncovered       12.0  0.722315  0.007215  0.711816   

                                     2.5%       50%     97.5%       max  
scenario          mam_treatment                                          
baseline          covered        1.000000  1.000000  1.000000  1.000000  
                  uncovered      0.698051  0.732792  0.767439  0.768510  
sqlns             covered        1.000000  1.000000  1.000000  1.000000  
                  uncovered      0.712130  0.722243  0.738025  0.739468  
wasting_treatment covered        1.000000  1.000000  1.000000  1.000000  
                  uncovered      0.712513  0.722229  0.735842  0.737586

In [156]:
# sam to mam (r2)
print('SAM to MAM RR target baseline:', rr_sam_to_mam_baseline,
      '\nSAM to MAM RR target alternative:', rr_sam_to_mam_alternative)
calculate_wasting_transition_rrs(wasting_transitions, wasting_pt, ages, 
                                 'severe_acute_malnutrition_to_moderate_acute_malnutrition_event_count', 
                                 'severe_acute_malnutrition', 
                                  ['input_draw','scenario','sam_treatment'])

SAM to MAM RR target baseline: 3.333333333333333 
SAM to MAM RR target alternative: 3.6363636363636362


count      mean       std       min  \
scenario          sam_treatment                                        
baseline          covered         12.0  1.000000  0.000000  1.000000   
                  uncovered       12.0  3.459278  0.379928  2.903834   
sqlns             covered         12.0  1.000000  0.000000  1.000000   
                  uncovered       12.0  3.701872  0.226830  3.364422   
wasting_treatment covered         12.0  1.000000  0.000000  1.000000   
                  uncovered       12.0  3.713336  0.222868  3.383590   

                                     2.5%       50%     97.5%       max  
scenario          sam_treatment                                          
baseline          covered        1.000000  1.000000  1.000000  1.000000  
                  uncovered      2.962210  3.444246  4.049090  4.095886  
sqlns             covered        1.000000  1.000000  1.000000  1.000000  
                  uncovered      3.400713  3.709955  4.072738  4.107935  
wasting_treatment covered        1.000000  1.000000  1.000000  1.000000  
                  uncovered      3.418062  3.709263  4.074729  4.103638

In [150]:
# sam to mild (t1)
print('SAM to mild RR target baseline:', rr_sam_to_mild_baseline,
      '\nSAM to mild RR target alternative:', rr_sam_to_mild_alternative)
calculate_wasting_transition_rrs(wasting_transitions, wasting_pt, ages, 
                                 'severe_acute_malnutrition_to_mild_child_wasting_event_count', 
                                 'severe_acute_malnutrition', 
                                  ['input_draw','scenario','sam_treatment'])

SAM to mild RR target baseline: 0 
SAM to mild RR target alternative: 0


count  mean  std  min  2.5%  50%  97.5%  max
scenario          sam_treatment                                              
baseline          covered         12.0   1.0  0.0  1.0   1.0  1.0    1.0  1.0
                  uncovered       12.0   0.0  0.0  0.0   0.0  0.0    0.0  0.0
sqlns             covered         12.0   1.0  0.0  1.0   1.0  1.0    1.0  1.0
                  uncovered       12.0   0.0  0.0  0.0   0.0  0.0    0.0  0.0
wasting_treatment covered         12.0   1.0  0.0  1.0   1.0  1.0    1.0  1.0
                  uncovered       12.0   0.0  0.0  0.0   0.0  0.0    0.0  0.0

# Step 2: verify that wasting incidence rates do not vary by treatment coverage

(note: this is a validation strategy we had imposed, but should perhaps be rethought?)

In [53]:
# i3 stratified by mam treatment
calculate_wasting_transition_rrs(wasting_transitions, wasting_pt, ages, 
                                 'susceptible_to_child_wasting_to_mild_child_wasting_event_count', 
                                 'susceptible_to_child_wasting', 
                                  ['input_draw','scenario','mam_treatment'])

count      mean       std       min  \
scenario          mam_treatment                                        
baseline          covered         12.0  1.000000  0.000000  1.000000   
                  uncovered       12.0  0.999939  0.002289  0.994854   
sqlns             covered         12.0  1.000000  0.000000  1.000000   
                  uncovered       12.0  0.999734  0.002661  0.995948   
wasting_treatment covered         12.0  1.000000  0.000000  1.000000   
                  uncovered       12.0  0.999804  0.002599  0.996250   

                                     2.5%       50%     97.5%       max  
scenario          mam_treatment                                          
baseline          covered        1.000000  1.000000  1.000000  1.000000  
                  uncovered      0.995477  1.000185  1.003138  1.003576  
sqlns             covered        1.000000  1.000000  1.000000  1.000000  
                  uncovered      0.996008  0.999242  1.004148  1.004510  
wasting_treatment covered        1.000000  1.000000  1.000000  1.000000  
                  uncovered      0.996351  0.999436  1.004041  1.004255

In [54]:
# i3 stratified by sam treatment
calculate_wasting_transition_rrs(wasting_transitions, wasting_pt, ages, 
                                 'susceptible_to_child_wasting_to_mild_child_wasting_event_count', 
                                 'susceptible_to_child_wasting', 
                                  ['input_draw','scenario','sam_treatment'])

count      mean       std       min  \
scenario          sam_treatment                                        
baseline          covered         12.0  1.000000  0.000000  1.000000   
                  uncovered       12.0  1.000702  0.001473  0.997788   
sqlns             covered         12.0  1.000000  0.000000  1.000000   
                  uncovered       12.0  1.001045  0.002076  0.997175   
wasting_treatment covered         12.0  1.000000  0.000000  1.000000   
                  uncovered       12.0  1.001077  0.002053  0.997099   

                                     2.5%       50%     97.5%       max  
scenario          sam_treatment                                          
baseline          covered        1.000000  1.000000  1.000000  1.000000  
                  uncovered      0.998209  1.000512  1.002787  1.002858  
sqlns             covered        1.000000  1.000000  1.000000  1.000000  
                  uncovered      0.997611  1.001362  1.003955  1.004368  
wasting_treatment covered        1.000000  1.000000  1.000000  1.000000  
                  uncovered      0.997528  1.001660  1.003905  1.004240

In [55]:
# i2 stratified by mam treatment
calculate_wasting_transition_rrs(wasting_transitions, wasting_pt, ages, 
                                 'mild_child_wasting_to_moderate_acute_malnutrition_event_count', 
                                 'mild_child_wasting', 
                                  ['input_draw','scenario','mam_treatment'])

count      mean       std       min  \
scenario          mam_treatment                                        
baseline          covered         12.0  1.000000  0.000000  1.000000   
                  uncovered       12.0  1.001503  0.005102  0.991812   
sqlns             covered         12.0  1.000000  0.000000  1.000000   
                  uncovered       12.0  1.068621  0.010499  1.052385   
wasting_treatment covered         12.0  1.000000  0.000000  1.000000   
                  uncovered       12.0  1.007020  0.004831  1.001201   

                                     2.5%       50%     97.5%       max  
scenario          mam_treatment                                          
baseline          covered        1.000000  1.000000  1.000000  1.000000  
                  uncovered      0.993541  1.001312  1.010794  1.011697  
sqlns             covered        1.000000  1.000000  1.000000  1.000000  
                  uncovered      1.053186  1.068011  1.085293  1.087025  
wasting_treatment covered        1.000000  1.000000  1.000000  1.000000  
                  uncovered      1.001384  1.005754  1.015214  1.015929

In [56]:
# i2 stratified by sam treatment
calculate_wasting_transition_rrs(wasting_transitions, wasting_pt, ages, 
                                 'mild_child_wasting_to_moderate_acute_malnutrition_event_count', 
                                 'mild_child_wasting', 
                                  ['input_draw','scenario','sam_treatment'])

count      mean       std       min  \
scenario          sam_treatment                                        
baseline          covered         12.0  1.000000  0.000000  1.000000   
                  uncovered       12.0  1.002113  0.003775  0.996307   
sqlns             covered         12.0  1.000000  0.000000  1.000000   
                  uncovered       12.0  1.028363  0.008115  1.019213   
wasting_treatment covered         12.0  1.000000  0.000000  1.000000   
                  uncovered       12.0  1.004536  0.003475  0.998160   

                                     2.5%       50%     97.5%       max  
scenario          sam_treatment                                          
baseline          covered        1.000000  1.000000  1.000000  1.000000  
                  uncovered      0.996840  1.000903  1.008451  1.008472  
sqlns             covered        1.000000  1.000000  1.000000  1.000000  
                  uncovered      1.019309  1.027121  1.042047  1.042698  
wasting_treatment covered        1.000000  1.000000  1.000000  1.000000  
                  uncovered      0.998895  1.005650  1.009376  1.009737

In [57]:
# i1 stratified by sam treatment
calculate_wasting_transition_rrs(wasting_transitions, wasting_pt, ages, 
                                 'moderate_acute_malnutrition_to_severe_acute_malnutrition_event_count', 
                                 'moderate_acute_malnutrition', 
                                  ['input_draw','scenario','mam_treatment'])

count      mean       std       min  \
scenario          mam_treatment                                        
baseline          covered         12.0  1.000000  0.000000  1.000000   
                  uncovered       12.0  0.997987  0.006971  0.987955   
sqlns             covered         12.0  1.000000  0.000000  1.000000   
                  uncovered       12.0  1.009715  0.006790  1.001435   
wasting_treatment covered         12.0  1.000000  0.000000  1.000000   
                  uncovered       12.0  1.011713  0.006907  1.002973   

                                     2.5%       50%     97.5%       max  
scenario          mam_treatment                                          
baseline          covered        1.000000  1.000000  1.000000  1.000000  
                  uncovered      0.988726  0.998973  1.008985  1.009968  
sqlns             covered        1.000000  1.000000  1.000000  1.000000  
                  uncovered      1.001620  1.009200  1.022025  1.023962  
wasting_treatment covered        1.000000  1.000000  1.000000  1.000000  
                  uncovered      1.003100  1.011096  1.024340  1.026263

# Step 3: Verifiy remission rates

In [58]:
# artifact
from vivarium import Artifact
art = Artifact('/ihme/costeffectiveness/artifacts/vivarium_ciff_sam/ethiopia.hdf',
               filter_terms=['year_start == 2020', f'age_end <= 5'])

In [78]:
def calculate_wasting_transition_rates(wasting_transitions, wasting_pt,
                                      ages, transition, source_state, 
                                      groupby_cols):
    transition = (wasting_transitions.loc[(wasting_transitions.age.isin(ages))
                                         & (wasting_transitions.measure==transition)]
                  .groupby(groupby_cols).sum())
    pt = (wasting_pt.loc[(wasting_pt.age.isin(ages))
                       & (wasting_pt.cause==source_state)]
          .groupby(groupby_cols).sum())
    transition_rate = 365 / (transition / pt)
    transition_rate = (transition_rate.groupby([c for c in groupby_cols if c != 'input_draw'])
                       .describe(percentiles=[0.025,0.975]))['value']
    return transition_rate

In [79]:
#r4 (30 days)
calculate_wasting_transition_rates(wasting_transitions, wasting_pt, ages, 
                                   'mild_child_wasting_to_susceptible_to_child_wasting_event_count', 
                                   'mild_child_wasting', 
                                      ['input_draw','scenario'])

,count,mean,std,min,2.5%,50%,97.5%,max
scenario,,,,,,,,
baseline,12.0,29.721623,0.026310,29.683731,29.684603,29.730404,29.757882,29.762796
sqlns,12.0,29.718298,0.028569,29.673875,29.675521,29.727446,29.757033,29.761452
wasting_treatment,12.0,29.719767,0.026890,29.679180,29.681159,29.726658,29.756436,29.760920


In [80]:
# r3 
# blend of 63 days untreated and 41 days treated
# looks reasonable
calculate_wasting_transition_rates(wasting_transitions, wasting_pt, ages, 
                                   'moderate_acute_malnutrition_to_mild_child_wasting_event_count', 
                                   'moderate_acute_malnutrition', 
                                      ['input_draw','scenario'])

,count,mean,std,min,2.5%,50%,97.5%,max
scenario,,,,,,,,
baseline,12.0,60.012739,0.750323,58.804160,58.909057,59.863399,61.239740,61.297961
sqlns,12.0,54.890663,0.480334,54.335579,54.340788,54.928611,55.639804,55.680703
wasting_treatment,12.0,54.655840,0.453651,54.049411,54.079609,54.700610,55.322488,55.341138


In [81]:
# r3
# blend of 63 days untreated and 41 days treated
# looks reasonable
calculate_wasting_transition_rates(wasting_transitions, wasting_pt, ages, 
                                   'moderate_acute_malnutrition_to_mild_child_wasting_event_count', 
                                   'moderate_acute_malnutrition', 
                                      ['input_draw','scenario','mam_treatment'])

count       mean       std        min  \
scenario          mam_treatment                                          
baseline          covered         12.0  46.271819  1.403786  43.914885   
                  uncovered       12.0  63.165286  0.130155  62.869995   
sqlns             covered         12.0  45.683747  0.507614  45.010550   
                  uncovered       12.0  63.198775  0.256555  62.847392   
wasting_treatment covered         12.0  45.660454  0.462132  45.029353   
                  uncovered       12.0  63.214367  0.233382  62.884071   

                                      2.5%        50%      97.5%        max  
scenario          mam_treatment                                              
baseline          covered        44.080250  46.291173  48.412997  48.449715  
                  uncovered      62.920732  63.160993  63.361029  63.364654  
sqlns             covered        45.026003  45.584502  46.556504  46.587927  
                  uncovered      62.874748  63.190961  63.596355  63.615528  
wasting_treatment covered        45.036218  45.613430  46.427653  46.444812  
                  uncovered      62.907792  63.196797  63.525018  63.528925

In [115]:
# r2
# untreated SAM recovery to MAM 
print('Based on uncovered rate...\ncovered baseline target: ', 365/((365/57.5) * 0.3),
      '\ncovered alternative target: ',  365/((365/57.5) * 0.275))
calculate_wasting_transition_rates(wasting_transitions, wasting_pt, ages, 
                                   'severe_acute_malnutrition_to_moderate_acute_malnutrition_event_count', 
                                   'severe_acute_malnutrition', 
                                      ['input_draw','scenario','sam_treatment'])

Based on uncovered rate...
covered baseline target:  191.66666666666669 
covered alternative target:  209.0909090909091


count        mean        std         min  \
scenario          sam_treatment                                             
baseline          covered         12.0  199.556619  45.416246  141.984182   
                  uncovered       12.0   57.483131  10.259483   44.332889   
sqlns             covered         12.0  213.212954  42.348175  164.838898   
                  uncovered       12.0   57.500178  10.289377   44.320591   
wasting_treatment covered         12.0  213.783060  42.158830  165.803223   
                  uncovered       12.0   57.487897  10.298092   44.257539   

                                       2.5%         50%       97.5%  \
scenario          sam_treatment                                       
baseline          covered        145.000375  199.601789  283.653723   
                  uncovered       45.407645   54.147887   76.795328   
sqlns             covered        165.088518  210.781956  293.302138   
                  uncovered       45.429033   54.191544   76.999341   
wasting_treatment covered        165.858132  211.394668  293.508627   
                  uncovered       45.389061   54.129568   76.930967   

                                        max  
scenario          sam_treatment              
baseline          covered        299.357745  
                  uncovered       80.448411  
sqlns             covered        308.705589  
                  uncovered       80.753293  
wasting_treatment covered        308.787212  
                  uncovered       80.668561

In [114]:
# t1 
# time to recovery effectively treated SAM = 48.3
print('covered baseline target: ', 365 / (365 / 48.3 * 0.7),
      '\ncovered alternative target: ', 365 / (365 / 48.3 * 0.725))

calculate_wasting_transition_rates(wasting_transitions, wasting_pt, ages, 
                                   'severe_acute_malnutrition_to_mild_child_wasting_event_count', 
                                   'severe_acute_malnutrition', 
                                      ['input_draw','scenario','sam_treatment'])

covered baseline target:  69.0 
covered alternative target:  66.62068965517241


count       mean       std        min  \
scenario          sam_treatment                                          
baseline          covered         12.0  68.591715  3.044725  64.845801   
                  uncovered       12.0        inf       NaN        inf   
sqlns             covered         12.0  66.520794  1.428210  64.530287   
                  uncovered       12.0        inf       NaN        inf   
wasting_treatment covered         12.0  66.470635  1.368444  64.518981   
                  uncovered       12.0        inf       NaN        inf   

                                      2.5%        50%      97.5%        max  
scenario          sam_treatment                                              
baseline          covered        64.921415  67.991928  73.462761  74.247180  
                  uncovered            inf        inf        inf        inf  
sqlns             covered        64.636327  66.236236  68.808434  69.133622  
                  uncovered            inf        inf        inf        inf  
wasting_treatment covered        64.617716  66.042653  68.608341  68.897186  
                  uncovered            inf        inf        inf        inf

# Step 4: validate incidence rates

In [116]:
#i3 
calculate_wasting_transition_rates(wasting_transitions, wasting_pt, ages, 
                                   'susceptible_to_child_wasting_to_mild_child_wasting_event_count', 
                                   'susceptible_to_child_wasting', 
                                      ['input_draw','scenario'])

,count,mean,std,min,2.5%,50%,97.5%,max
scenario,,,,,,,,
baseline,12.0,101.989802,0.758337,99.908964,100.366012,102.192207,102.803435,102.924144
sqlns,12.0,101.971766,0.757605,99.887643,100.341031,102.172114,102.773805,102.893710
wasting_treatment,12.0,101.977830,0.758406,99.895221,100.349707,102.182948,102.790027,102.913304


In [118]:
#i2
calculate_wasting_transition_rates(wasting_transitions, wasting_pt, ages, 
                                   'mild_child_wasting_to_moderate_acute_malnutrition_event_count', 
                                   'mild_child_wasting', 
                                      ['input_draw','scenario'])

,count,mean,std,min,2.5%,50%,97.5%,max
scenario,,,,,,,,
baseline,12.0,160.486655,4.402687,155.016590,155.033648,159.706739,168.927192,169.484239
sqlns,12.0,174.063622,4.526199,170.165141,170.215190,172.113476,183.284899,184.242359
wasting_treatment,12.0,159.955161,4.340365,154.622205,154.642501,159.247063,168.290734,168.851250


In [121]:
#i2
calculate_wasting_transition_rates(wasting_transitions, wasting_pt, ages, 
                                   'moderate_acute_malnutrition_to_severe_acute_malnutrition_event_count', 
                                   'moderate_acute_malnutrition', 
                                      ['input_draw','scenario'])

,count,mean,std,min,2.5%,50%,97.5%,max
scenario,,,,,,,,
baseline,12.0,195.433992,17.306117,166.658496,170.506081,191.543278,221.987080,226.479743
sqlns,12.0,194.113221,17.207914,165.126728,169.043768,190.556575,220.553777,225.026847
wasting_treatment,12.0,194.826187,17.175172,166.125089,169.942300,191.270570,221.310176,225.806979


In [140]:
# overall transition out of SAM
print('baseline target: 6.7 (95% CI: 5.3-8.4)')
overall_sam = (wasting_transitions.loc[(wasting_transitions.measure.str.contains('severe_acute_malnutrition_to'))
                       & (wasting_transitions.age.isin(ages))]
               .groupby(['input_draw','scenario']).sum()).drop(columns='year')
deaths = pd.read_csv(output_dir + 'deaths.csv')
sam_deaths = deaths.loc[(deaths.wasting_state=='severe_acute_malnutrition')
                       &(deaths.age.isin(ages))].groupby(['input_draw','scenario']).sum()
sam_pt = (wasting_pt.loc[(wasting_pt.cause=='severe_acute_malnutrition')
                        & (wasting_pt.age.isin(ages))]
         .groupby(['input_draw','scenario']).sum())#['value']
overall_sam_remission = (((overall_sam + sam_deaths)/ sam_pt)).groupby('scenario').describe(percentiles=[0.025,0.975])['value']
overall_sam_remission

baseline target: 6.7 (95% CI: 5.3-8.4)


,count,mean,std,min,2.5%,50%,97.5%,max
scenario,,,,,,,,
baseline,12.0,6.876026,0.687064,5.533690,5.721291,6.991568,7.855919,7.961835
sqlns,12.0,6.959922,0.607378,5.797439,5.953345,7.072093,7.829758,7.921951
wasting_treatment,12.0,6.963337,0.604282,5.812116,5.965293,7.084688,7.833892,7.933162


In [141]:
365 / 6.7

54.47761194029851

In [142]:
365 / 6.87

53.12954876273653

# look at incidence rates stratified by x-factor exposure

In [143]:
#i3 
calculate_wasting_transition_rates(wasting_transitions, wasting_pt, ages, 
                                   'susceptible_to_child_wasting_to_mild_child_wasting_event_count', 
                                   'susceptible_to_child_wasting', 
                                      ['input_draw','scenario','x_factor'])

count        mean       std         min  \
scenario          x_factor                                            
baseline          cat1       12.0  102.179617  0.743278  100.076097   
                  cat2       12.0  101.824845  0.784103   99.763519   
sqlns             cat1       12.0  102.142692  0.741249  100.044415   
                  cat2       12.0  101.820722  0.783109   99.748991   
wasting_treatment cat1       12.0  102.156248  0.743932  100.049062   
                  cat2       12.0  101.821092  0.782089   99.760065   

                                  2.5%         50%       97.5%         max  
scenario          x_factor                                                  
baseline          cat1      100.573135  102.330593  102.865399  102.929762  
                  cat2      100.165746  101.992729  102.756159  102.919274  
sqlns             cat1      100.543907  102.302010  102.828592  102.888202  
                  cat2      100.156520  101.991795  102.736681  102.898572  
wasting_treatment cat1      100.552577  102.319810  102.853351  102.929108  
                  cat2      100.161101  101.993040  102.738472  102.899462

In [144]:
#i2 
calculate_wasting_transition_rates(wasting_transitions, wasting_pt, ages, 
                                   'mild_child_wasting_to_moderate_acute_malnutrition_event_count', 
                                   'mild_child_wasting', 
                                      ['input_draw','scenario','x_factor'])

count        mean       std         min  \
scenario          x_factor                                            
baseline          cat1       12.0  104.307291  2.844180  100.782513   
                  cat2       12.0  330.158758  9.087753  318.297339   
sqlns             cat1       12.0  113.628228  3.031927  111.100665   
                  cat2       12.0  360.017291  9.256446  350.948296   
wasting_treatment cat1       12.0  104.261068  2.842812  100.746883   
                  cat2       12.0  330.057803  9.079323  318.278893   

                                  2.5%         50%       97.5%         max  
scenario          x_factor                                                  
baseline          cat1      100.803414  103.898430  109.887451  110.316771  
                  cat2      318.711949  329.987251  346.920570  348.371668  
sqlns             cat1      111.136582  112.227188  119.800364  120.491918  
                  cat2      351.032634  357.088630  378.538545  380.739594  
wasting_treatment cat1      100.782234  103.885485  109.838011  110.253914  
                  cat2      318.656607  329.733388  346.841729  348.280065

In [146]:
#i1
calculate_wasting_transition_rates(wasting_transitions, wasting_pt, ages, 
                                   'moderate_acute_malnutrition_to_severe_acute_malnutrition_event_count', 
                                   'moderate_acute_malnutrition', 
                                      ['input_draw','scenario','x_factor'])

count        mean        std         min  \
scenario          x_factor                                             
baseline          cat1       12.0  162.424456  14.377628  138.140918   
                  cat2       12.0  527.642566  45.668894  450.802290   
sqlns             cat1       12.0  161.747187  14.367809  137.336757   
                  cat2       12.0  527.129743  46.301734  450.943467   
wasting_treatment cat1       12.0  162.018608  14.281060  137.901442   
                  cat2       12.0  526.710694  46.150447  449.570258   

                                  2.5%         50%       97.5%         max  
scenario          x_factor                                                  
baseline          cat1      141.629478  159.332628  184.474639  188.159207  
                  cat2      457.861954  518.232014  598.433704  607.826951  
sqlns             cat1      140.867332  158.923586  183.825437  187.530907  
                  cat2      457.515485  517.694193  600.602249  609.877708  
wasting_treatment cat1      141.357229  159.205180  184.073061  187.750015  
                  cat2      456.857149  516.226136  598.739973  608.187628

In [147]:
#i1
calculate_wasting_transition_rates(wasting_transitions, wasting_pt, ages, 
                                   'moderate_acute_malnutrition_to_severe_acute_malnutrition_event_count', 
                                   'moderate_acute_malnutrition', 
                                      ['input_draw','scenario','x_factor','mam_treatment'])

count        mean        std  \
scenario          x_factor mam_treatment                                 
baseline          cat1     covered         12.0  162.192375  13.665251   
                           uncovered       12.0  162.459215  14.510857   
                  cat2     covered         12.0  524.303118  57.738715   
                           uncovered       12.0  528.000078  44.759091   
sqlns             cat1     covered         12.0  161.568960  14.677582   
                           uncovered       12.0  161.874523  14.184943   
                  cat2     covered         12.0  527.770141  45.863012   
                           uncovered       12.0  526.748434  48.094757   
wasting_treatment cat1     covered         12.0  161.954074  14.515003   
                           uncovered       12.0  162.074694  14.143876   
                  cat2     covered         12.0  527.322138  46.255006   
                           uncovered       12.0  526.231045  46.985856   

                                                 min        2.5%         50%  \
scenario          x_factor mam_treatment                                       
baseline          cat1     covered        138.903824  142.222761  159.833591   
                           uncovered      137.963252  141.456486  159.229108   
                  cat2     covered        451.952799  452.628355  514.963003   
                           uncovered      450.533536  458.781822  522.308992   
sqlns             cat1     covered        136.358788  140.035883  158.519126   
                           uncovered      138.034964  141.359486  159.169285   
                  cat2     covered        460.234837  464.157820  521.636901   
                           uncovered      444.185290  452.681910  521.729899   
wasting_treatment cat1     covered        137.400304  140.876180  159.005784   
                           uncovered      138.279118  141.554425  159.340632   
                  cat2     covered        455.973157  461.993389  516.008283   
                           uncovered      444.545051  452.850052  523.431733   

                                               97.5%         max  
scenario          x_factor mam_treatment                          
baseline          cat1     covered        183.213202  186.759453  
                           uncovered      184.668180  188.364084  
                  cat2     covered        625.476428  636.078761  
                           uncovered      596.004304  603.937209  
sqlns             cat1     covered        184.926596  188.807286  
                           uncovered      183.338028  186.751420  
                  cat2     covered        590.419518  596.324592  
                           uncovered      607.913113  619.167926  
wasting_treatment cat1     covered        185.152211  188.885453  
                           uncovered      183.558243  187.019953  
                  cat2     covered        595.309903  603.095073  
                           uncovered      601.158975  611.799926